In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import preprocessor as p
import random
import csv
import os
import ast
import argparse
import numpy as np
import pandas as pd
import warnings
import import_ipynb
import preprocessing as pp
import data_helper as dh
# from .data_helper import load_dataset
import modeling
import modeling, metrics, model_utils,data_helper
import importlib

importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from data_helper.ipynb
importing Jupyter notebook from modeling.ipynb
importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from model_utils.ipynb


In [8]:
warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [9]:
args = {}

# Assign values directly to the arguments
args['config_file'] = r'C:\Users\HP\Documents\pk\TSE\config\config-bertweet.txt'
args['seed'] = 1
args['model_select'] = 'bertweet'
args['model_dir'] = r'C:\Users\HP\Documents\pk\TSE\mode_dir'
args['train_data'] = r'C:\Users\HP\Documents\pk\TSE\data\train.csv'
args['dev_data'] = r'C:\Users\HP\Documents\pk\TSE\data\val.csv'
args['test_data'] = r'C:\Users\HP\Documents\pk\TSE\data\test.csv'
args['aux_eval'] = False  # Set to True or False based on your requirement
args['mul_task'] = False  # Set to True or False based on your requirement

In [10]:
print(args['model_select'])

bertweet


In [13]:
# gpu or cpumode
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = "cpu"

# load config file
with open(args['config_file'], 'r') as f:
    config = dict()
    for l in f.readlines():
        config[l.strip().split(":")[0]] = l.strip().split(":")[1]
print(device)

cpu


In [14]:
# print parameters in the log file
random_seeds = []
random_seeds.append(int(args['seed']))
outdir = args['model_dir']
model_select = args['model_select']
mul_task = args['mul_task']
batch_size = int(config['batch_size'])
print("Model: ",model_select)
print("Batch size: ",config['batch_size'])
print("Multi-task learning: ",args['mul_task'])
print(60*"#")


Model:  bertweet
Batch size:  128
Multi-task learning:  False
############################################################


In [15]:
# load train/val/test sets
file = [args['train_data'], args['dev_data'], args['test_data']]
print(outdir, file[0], file[1], file[2],sep="\n")
print("\n\n\n\n\n---pk------\n")

C:\Users\HP\Documents\pk\TSE\mode_dir
C:\Users\HP\Documents\pk\TSE\data\train.csv
C:\Users\HP\Documents\pk\TSE\data\val.csv
C:\Users\HP\Documents\pk\TSE\data\test.csv





---pk------



In [16]:
print("load_dataset is started")
if model_select.startswith('bert'):
    print('\n1')
    x_train_all, x_val_all, x_test_all, x_train_aux_all, _ = data_helper.load_dataset(file, model_select, config)
else:
    print('\n2')
    x_train_all, x_val_all, x_test_all, x_train_aux_all, _, word_vectors = data_helper.load_dataset(file, model_select, config)
split_point = len(x_train_all[0])

if mul_task:
    print('\n3')
    x_train_all = [a + b for a, b in zip(x_train_all,x_train_aux_all)]
print("load_dataset is ended")

load_dataset is started

1
data size in main task:  42258
data size in auxiliary task:  29747
data size in main task:  5394
data size in auxiliary task:  3886
data size in main task:  9146
data size in auxiliary task:  5804
Length of the set: 42258
Length of the set: 5394
Length of the set: 9146
Length of the set: 29747
load_dataset is ended


In [20]:
# import random
# import numpy as np

# # Function to sample a subset of the data
# def sample_subset(data, size):
#     # Ensure each element in data can be indexed (e.g., numpy arrays, tensors)
#     indices = list(range(len(data[0])))
#     random.shuffle(indices)
#     subset_indices = indices[:size]
#     return [[d[i] for i in subset_indices] for d in data]

# # Set a reduced dataset size for testing
# # subset_size = 1000  # Adjust this number based on your needs

# # Sample subsets for training, validation, and test
# x_train_all = sample_subset(x_train_all,size)
# x_val_all = sample_subset(x_val_all, size // 5)  # 1/5 of training subset size
# x_test_all = sample_subset(x_test_all, size // 5)

# print("Sampled subset sizes:")
# print("Training set:", len(x_train_all[0]))
# print("Validation set:", len(x_val_all[0]))
# print("Test set:", len(x_test_all[0]))


In [21]:
print(len(x_train_all[0]))

42258


In [22]:
print("data loader is started")
if model_select.startswith('bert'):
    print('\n1')
    _, _, _, y_train, _, trainloader = dh.data_loader(x_train_all, batch_size, 'train', model_select, mul_task, split_point)
    _, _, _, y_val, _, valloader = dh.data_loader(x_val_all, batch_size, 'val', model_select)   
    _, _, _, y_test, _, testloader = dh.data_loader(x_test_all, batch_size, 'test', model_select)   
else:
    print('\n2')
    _, y_train, _, _, trainloader = dh.data_loader(x_train_all, batch_size, 'train', model_select, mul_task, split_point)
    _, y_val, _, _, valloader = dh.data_loader(x_val_all, batch_size, 'val', model_select) 
    _, y_test, _, _, testloader = dh.data_loader(x_test_all, batch_size, 'test', model_select) 
y_val = y_val.to(device)
y_test = y_test.to(device)

print("data loader is ended")

data loader is started

1
data loader is ended


In [ ]:
import os
# Directory to save checkpoints
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Helper function to find the latest checkpoint
def find_latest_checkpoint(checkpoint_dir, model_select, seed):
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith(f'{model_select}_seed{seed}_epoch')]
    if checkpoint_files:
        epoch_numbers = [int(f.split('_')[-1].split('.')[0]) for f in checkpoint_files]
        latest_epoch = max(epoch_numbers)
        latest_checkpoint = f'{model_select}_seed{seed}_epoch_{latest_epoch}.pt'
        return os.path.join(checkpoint_dir, latest_checkpoint), latest_epoch
    else:
        return None, 0

# Helper function to load checkpoint
def load_checkpoint(checkpoint_path, model, optimizer):
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
        return start_epoch
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0

# Helper function to save checkpoint
def save_checkpoint(checkpoint_dir, model, optimizer, model_select, seed, epoch):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    checkpoint_path = os.path.join(checkpoint_dir, f'{model_select}_seed{seed}_epoch_{epoch}.pt')
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

In [30]:
from tqdm import tqdm

In [ ]:
# Test with multiple random seeds
for seed in random_seeds:
    print("Current random seed:", seed)
    print("multitasking:", mul_task)

    # Set up the random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Model setup
    print('\nModel setup is started')
    model, optimizer = model_utils.model_setup(config, model_select, device)

    if model_select not in ['bert', 'bertweet']:
        print('\nSetting up embeddings')
        et = torch.tensor(list(word_vectors.values()), dtype=torch.float32)
        model.embedding.weight = torch.nn.Parameter(et, requires_grad=False)

    loss_function = torch.nn.CrossEntropyLoss()
    kwargs = {
        "model": model,
        "optimizer": optimizer,
        "model_select": model_select,
    }
    updater = model_utils.model_updater(**kwargs)  # Use the enhanced model_updater

    best_val = 0
    best_test_micro, best_test_macro = [], []

    # Find the latest checkpoint for this seed
    latest_checkpoint, start_epoch = find_latest_checkpoint(checkpoint_dir, model_select, seed)
    start_epoch = load_checkpoint(latest_checkpoint, model, optimizer) if latest_checkpoint else 0

    # Epoch loop with tqdm
    total_epochs = int(config['total_epochs'])
    for epoch in tqdm(range(start_epoch, total_epochs), desc="Epochs", unit="epoch"):
        print('Epoch:', epoch)

        # Train
        print('Training phase started')
        updater.model.train()

        try:
            # Add more detailed debugging information
            print('Starting update function...')
            sum_loss = updater.update(trainloader, loss_function, device, max_batches=100)
            print(f'Completed update function, Training loss: {sum_loss / len(y_train):.4f}')

            # Save checkpoint after each epoch
            save_checkpoint(checkpoint_dir, updater.model, optimizer, model_select, seed, epoch + 1)

        except Exception as e:
            print(f"Exception in updater.update: {e}")
            continue  # Continue to the next epoch

        # Evaluation on validation set
        print('Validation phase started')
        updater.model.eval()
        with torch.no_grad():
            preds = model_utils.model_preds(valloader, updater.model, device, model_select)
            f1_average = metrics.train_compute_f1(preds, y_val)
            print(f'Validation F1: {f1_average:.4f}')

        if f1_average > best_val:
            best_val = f1_average
            model_weight = os.path.join(outdir, f'{model_select}_seed{seed}.pt')
            torch.save(updater.model.state_dict(), model_weight)
            print(f'Model weights saved to {model_weight}')

    print(f"Best validation results of model {model_select} and seed {seed}: {best_val:.4f}")

    # Evaluation on test set
    print('Test phase started')
    weight = os.path.join(outdir, f'{model_select}_seed{seed}.pt')
    model.load_state_dict(torch.load(weight, map_location=device))

    model.eval()
    with torch.no_grad():
        preds = model_utils.model_preds(testloader, model, device, model_select)

        # Micro-averaged F1
        f1_average = metrics.train_compute_f1(preds, y_test)
        best_test_micro.append(f1_average)
        print(f'Micro-averaged F1: {f1_average:.4f}')

        # Macro-averaged F1
        preds_list = dh.sep_test_set(preds)
        y_test_list = dh.sep_test_set(y_test)
        temp_list = []
        for ind in range(len(y_test_list)):
            f1_average = metrics.train_compute_f1(preds_list[ind], y_test_list[ind])
            temp_list.append(f1_average)
            print(f'Macro-averaged F1 for test set {ind}: {f1_average:.4f}')

        best_test_macro.append(sum(temp_list) / len(temp_list))

    print("Best micro test results:", ", ".join(f'{x:.4f}' for x in best_test_micro))
    print("Best micro test results on SemEval-2016:", f'{temp_list[0]:.4f}')
    print("Best micro test results on COVID-19:", f'{temp_list[1]:.4f}')
    print("Best micro test results on argmin:", f'{temp_list[2]:.4f}')
    print("Best micro test results on PStance:", f'{temp_list[3]:.4f}')
    print("Best macro test results:", ", ".join(f'{x:.4f}' for x in best_test_macro))

Current random seed: 1
multitasking: False

Model setup is started
Loading BERT model: bertweet
Loading BERTweet model.
Checkpoint loaded. Resuming from epoch 1


Epochs:   0%|                                                                                 | 0/4 [00:00<?, ?epoch/s]

Epoch: 1
Training phase started
Starting update function...
len(trainloader)= 331
Total batches to be processed: 100
Using BERT-based model for updating
Processing batch 1
Batch loss: 1.0793
Processing batch 2
Batch loss: 1.1022
Processing batch 3
Batch loss: 1.1372
Processing batch 4
Batch loss: 1.0951
Processing batch 5
Batch loss: 1.1276
Processing batch 6
Batch loss: 1.0928
Processing batch 7
Batch loss: 1.0663
Processing batch 8
Batch loss: 1.0590
Processing batch 9
Batch loss: 1.0549
Processing batch 10
Batch loss: 1.1068
Processing batch 11
Batch loss: 1.0523
Processing batch 12
Batch loss: 1.0398
Processing batch 13
Batch loss: 0.9657
Processing batch 14
Batch loss: 0.9514
Processing batch 15
Batch loss: 0.9235
Processing batch 16
Batch loss: 0.9608
Processing batch 17


In [27]:
# # Dummy classes/functions for demonstration. Replace these with actual implementations.
# # from your_module import model_utils, metrics, dh 

# # Test with multiple random seeds
# for seed in random_seeds:    
#     print("Current random seed:", seed)
#     print("mutlitasking:",mul_task)
#     # Set up the random seed
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     # No CUDA seed setup as we are running on CPU

#     # Model setup
#     print('\nModel setup is started')
#     model, optimizer = model_utils.model_setup(config, model_select, device)
    
#     if model_select not in ['bert', 'bertweet']:
#         print('\nSetting up embeddings')
#         et = torch.tensor(list(word_vectors.values()), dtype=torch.float32)
#         model.embedding.weight = torch.nn.Parameter(et, requires_grad=False)
    
#     loss_function = torch.nn.CrossEntropyLoss()
#     kwargs = {
#         "model": model,
#         "optimizer": optimizer,
#         "model_select": model_select,
#     }
#     updater = model_utils.model_updater(**kwargs)  # Use the enhanced model_updater
    
#     best_val = 0
#     best_test_micro, best_test_macro = [], []
    
# #     for epoch in range(0, int(config['total_epochs'])):
#         # Epoch loop with tqdm
#     total_epochs = int(config['total_epochs'])
#     for epoch in tqdm(range(total_epochs), desc="Epochs", unit="epoch"):
#         print('Epoch:', epoch)

#         # Train
#         print('Training phase started')
#         updater.model.train()
        
#         try:
#             # Add more detailed debugging information
#             print('Starting update function...')
#             sum_loss = updater.update(trainloader, loss_function, device,max_batches=100)
#             print(f'Completed update function, Training loss: {sum_loss/len(y_train):.4f}')

#         except Exception as e:
#             print(f"Exception in updater.update: {e}")
#             continue  # Continue to the next epoch
        
#         # Evaluation on validation set
#         print('Validation phase started')
#         updater.model.eval()
#         with torch.no_grad():
#             preds = model_utils.model_preds(valloader, updater.model, device, model_select)
#             f1_average = metrics.train_compute_f1(preds, y_val)
#             print(f'Validation F1: {f1_average:.4f}')
        
#         if f1_average > best_val:
#             best_val = f1_average
#             model_weight = os.path.join(outdir, f'{model_select}_seed{seed}.pt')
#             torch.save(updater.model.state_dict(), model_weight)
#             print(f'Model weights saved to {model_weight}')
    
#     print(f"Best validation results of model {model_select} and seed {seed}: {best_val:.4f}")
    
#     # Evaluation on test set
#     print('Test phase started')
#     weight = os.path.join(outdir, f'{model_select}_seed{seed}.pt')
#     model.load_state_dict(torch.load(weight, map_location=device))
    
#     model.eval()
#     with torch.no_grad():
#         preds = model_utils.model_preds(testloader, model, device, model_select)

#         # Micro-averaged F1
#         f1_average = metrics.train_compute_f1(preds, y_test)
#         best_test_micro.append(f1_average)
#         print(f'Micro-averaged F1: {f1_average:.4f}')
        
#         # Macro-averaged F1
#         preds_list = dh.sep_test_set(preds) 
#         y_test_list = dh.sep_test_set(y_test)
#         temp_list = []
#         for ind in range(len(y_test_list)):
#             f1_average = metrics.train_compute_f1(preds_list[ind], y_test_list[ind])
#             temp_list.append(f1_average)
#             print(f'Macro-averaged F1 for test set {ind}: {f1_average:.4f}')
        
#         best_test_macro.append(sum(temp_list)/len(temp_list))
            
#     print("Best micro test results:", ", ".join(f'{x:.4f}' for x in best_test_micro))
#     print("Best micro test results on SemEval-2016:", f'{temp_list[0]:.4f}')
#     print("Best micro test results on COVID-19:", f'{temp_list[1]:.4f}')
#     print("Best micro test results on argmin:", f'{temp_list[2]:.4f}')
#     print("Best micro test results on PStance:", f'{temp_list[3]:.4f}')
#     print("Best macro test results:", ", ".join(f'{x:.4f}' for x in best_test_macro))


In [1]:
temp_list

In [ ]:
# def train():
    
#     # parser = argparse.ArgumentParser()
#     # parser.add_argument('-c', '--config_file', help='Name of the cofig data file', required=False)
#     # parser.add_argument('-s', '--seed', help='Random seed', required=False)
#     # parser.add_argument('-m', '--model_select', help='Model name', required=False)
#     # parser.add_argument('-mod_dir', '--model_dir', help='Saved model dir', required=False)
#     # parser.add_argument('-train', '--train_data', help='Name of the train data file', default=None, required=False)
#     # parser.add_argument('-dev', '--dev_data', help='Name of the dev data file', default=None, required=False)
#     # parser.add_argument('-test', '--test_data', help='Name of the test data file', default=None, required=False)
#     # parser.add_argument('-a', '--aux_eval', help='Auxiliary task', action='store_true')
#     # parser.add_argument('-mul', '--mul_task', help='Multi-task with target prediction as aux task', action='store_true')
#     # args = vars(parser.parse_args())

#     args = {}
    
#     # Assign values directly to the arguments
#     args['config_file'] = r'C:\Users\n1900\Documents\TSE\config\config-bertweet.txt'
#     args['seed'] = 42
#     args['model_select'] = 'bert'
#     args['model_dir'] = 'path/to/model_directory'
#     args['train_data'] = r'C:\Users\n1900\Documents\TSE\data\train.csv'
#     args['dev_data'] = r'C:\Users\n1900\Documents\TSE\data\val.csv'
#     args['test_data'] = r'C:\Users\n1900\Documents\TSE\data\test.csv'
#     args['aux_eval'] = False  # Set to True or False based on your requirement
#     args['mul_task'] = False  # Set to True or False based on your requirement
    

#     # gpu or cpumode
#     if torch.cuda.is_available():
#         device = torch.device("cuda")
#     else:
#         device = "cpu"
    
#     # load config file
#     with open(args['config_file'], 'r') as f:
#         config = dict()
#         for l in f.readlines():
#             config[l.strip().split(":")[0]] = l.strip().split(":")[1]
    
#     # print parameters in the log file
#     random_seeds = []
#     random_seeds.append(int(args['seed']))
#     outdir = args['model_dir']
#     model_select = args['model_select']
#     mul_task = args['mul_task']
#     batch_size = int(config['batch_size'])
#     print("Model: ",model_select)
#     print("Batch size: ",config['batch_size'])
#     print("Multi-task learning: ",args['mul_task'])
#     print(60*"#")
    
#     # load train/val/test sets
#     file = [args['train_data'], args['dev_data'], args['test_data']]
#     print(outdir, file[0], file[1], file[2],sep="\n")
#     print("\n\n\n\n\n---pk------\n")
#     print("load_dataset is started")
#     if model_select.startswith('bert'):
#         x_train_all, x_val_all, x_test_all, x_train_aux_all, _ = load_dataset(file, model_select, config)
#     else:
#         x_train_all, x_val_all, x_test_all, x_train_aux_all, _, word_vectors = load_dataset(file, model_select, config)
#     split_point = len(x_train_all[0])
    
#     if mul_task:
#         x_train_all = [a + b for a, b in zip(x_train_all,x_train_aux_all)]
#     print("load_dataset is ended")

#     print("data loader is started")
#     if model_select.startswith('bert'):
#         _, _, _, y_train, _, trainloader = dh.data_loader(x_train_all, batch_size, 'train', model_select, mul_task, split_point)
#         _, _, _, y_val, _, valloader = dh.data_loader(x_val_all, batch_size, 'val', model_select)   
#         _, _, _, y_test, _, testloader = dh.data_loader(x_test_all, batch_size, 'test', model_select)   
#     else:
#         _, y_train, _, _, trainloader = dh.data_loader(x_train_all, batch_size, 'train', model_select, mul_task, split_point)
#         _, y_val, _, _, valloader = dh.data_loader(x_val_all, batch_size, 'val', model_select) 
#         _, y_test, _, _, testloader = dh.data_loader(x_test_all, batch_size, 'test', model_select) 
#     y_val = y_val.to(device)
#     y_test = y_test.to(device)

#     print("data loader is ended")
#     # test
#     for seed in random_seeds:    
#         print("current random seed: ", seed)
        
#         # set up the random seed
#         random.seed(seed)
#         np.random.seed(seed)
#         torch.manual_seed(seed) 
#         torch.cuda.manual_seed(seed)
#         torch.cuda.manual_seed_all(seed)
#         torch.backends.cudnn.benchmark = False
#         torch.backends.cudnn.deterministic = True
        
#         # model setup
#         model, optimizer = model_utils.model_setup(config, model_select, device)
#         if model_select not in ['bert','bertweet']:
#             et = torch.tensor(list(word_vectors.values()), dtype=torch.float32).cuda()
#             model.embedding.weight = nn.Parameter(et, requires_grad = False)
#         loss_function = nn.CrossEntropyLoss()
#         kwargs = {
#                     "model": model,
#                     "optimizer": optimizer,
#                     "model_select": model_select,
#         }
#         updater = model_utils.model_updater(**kwargs)
        
#         best_val = 0
#         best_test_micro, best_test_macro = [], []
#         for epoch in range(0, int(config['total_epochs'])):
#             print('Epoch:', epoch)

#             # train
#             print(
#             updater.model.train()
#             print('1')
#             sum_loss = updater.update(trainloader, loss_function, device)
#             print('2')
#             print(sum_loss/len(y_train))

#             # evaluation on validation set
#             updater.model.eval()
#             with torch.no_grad():
#                 preds = model_utils.model_preds(valloader, updater.model, device, model_select)
#                 f1_average = metrics.train_compute_f1(preds, y_val)
                
#             if f1_average > best_val:
#                 best_val = f1_average
#                 model_weight = os.path.join(outdir,model_select+'_seed{}.pt'.format(seed))
#                 torch.save(updater.model.state_dict(), model_weight)

#         print("Best val results of model {} and seed {} are: {}".format(model_select, seed, best_val))
        
#         # evaluation on test set 
#         weight = os.path.join(outdir, model_select+'_seed{}.pt'.format(seed))
#         model.load_state_dict(torch.load(weight))

#         model.eval()
#         with torch.no_grad():
#             preds = model_utils.model_preds(testloader, model, device, model_select)

#             # micro-averaged F1
#             f1_average = metrics.train_compute_f1(preds, y_test)
#             best_test_micro.append(f1_average)

#             # macro-averaged F1
#             preds_list = dh.sep_test_set(preds) 
#             y_test_list = dh.sep_test_set(y_test)
#             temp_list = []
#             for ind in range(len(y_test_list)):
#                 f1_average = metrics.train_compute_f1(preds_list[ind], y_test_list[ind])
#                 temp_list.append(f1_average)
#             best_test_macro.append(sum(temp_list)/len(temp_list))
                
#         print("Best micro test results: " + ",".join(map(str, best_test_micro)))
#         print("Best micro test results on SemEval-2016: " + ",".join(map(str, [temp_list[0]])))
#         print("Best micro test results on COVID-19: " + ",".join(map(str, [temp_list[1]])))
#         print("Best micro test results on argmin: " + ",".join(map(str, [temp_list[2]])))
#         print("Best micro test results on PStance: " + ",".join(map(str, [temp_list[3]])))
#         print("Best macro test results: " + ",".join(map(str, best_test_macro)))
    
# if __name__ == "__main__":
#     train()

Model:  bert
Batch size:  64
Multi-task learning:  False
############################################################
path/to/model_directory
C:\Users\n1900\Documents\TSE\data\train.csv
C:\Users\n1900\Documents\TSE\data\val.csv
C:\Users\n1900\Documents\TSE\data\test.csv





---pk------

load_dataset is started
data size in main task:  42258
data size in auxiliary task:  29747
data size in main task:  5394
data size in auxiliary task:  3886
data size in main task:  9146
data size in auxiliary task:  5804
Length of the set: 42258
Length of the set: 5394
Length of the set: 9146
Length of the set: 29747
load_dataset is ended
data loader is started
data loader is ended
current random seed:  42
Epoch: 0
